In [0]:
%sql
USE CATALOG workspace;
USE SCHEMA default;

In [0]:
# --- Konfiguration ---
volume_path = "/Volumes/workspace/default/volume"
source_data_path = f"{volume_path}/enrollments_raw" # Eigener Ordner für die Quelldateien

# Pfade für die Checkpoints jeder Schicht
chk_bronze = f"{volume_path}/_chk/medallion_bronze"
chk_silver = f"{volume_path}/_chk/medallion_silver"
chk_gold = f"{volume_path}/_chk/medallion_gold"

# Altkram löschen

In [0]:
# --- Aufräumen ---
# 1. Tabellen löschen
spark.sql("DROP TABLE IF EXISTS students")
spark.sql("DROP TABLE IF EXISTS enrollments")
spark.sql("DROP TABLE IF EXISTS enrollments_bronze")
spark.sql("DROP TABLE IF EXISTS enrollments_silver")
spark.sql("DROP TABLE IF EXISTS daily_student_courses")
print("✅ Alte Tabellen gelöscht.")

# 2. Checkpoints und Quelldaten-Ordner löschen
dbutils.fs.rm(source_data_path, recurse=True)
dbutils.fs.rm(chk_bronze, recurse=True)
dbutils.fs.rm(chk_silver, recurse=True)
dbutils.fs.rm(chk_gold, recurse=True)
print("✅ Alte Checkpoints und Quelldaten gelöscht.")

✅ Alte Tabellen gelöscht.
✅ Alte Checkpoints und Quelldaten gelöscht.


# Ausgangssituation erstellen

In [0]:
%python
dataset_school = "/Volumes/workspace/default/volume"

all_files = dbutils.fs.ls(dataset_school)
json_files = [f for f in all_files if f.name.endswith(".json")]

display(json_files)

path,name,size,modificationTime
dbfs:/Volumes/workspace/default/volume/courses.json,courses.json,329,1754654637000
dbfs:/Volumes/workspace/default/volume/courses_part2.json,courses_part2.json,348,1754991776000
dbfs:/Volumes/workspace/default/volume/enrollments.json,enrollments.json,371,1754654637000
dbfs:/Volumes/workspace/default/volume/nested_json.json,nested_json.json,1186,1754663219000
dbfs:/Volumes/workspace/default/volume/students.json,students.json,322,1754654637000
dbfs:/Volumes/workspace/default/volume/students_update.json,students_update.json,216,1754726175000


In [0]:
# JSON lesen:
enrollments_df = spark.read.json("/Volumes/workspace/default/volume/enrollments.json")
enrollments_df.createOrReplaceTempView("enrollments")

In [0]:
# DataFrame als neue JSON-Dateien in den überwachten Ordner schreiben.
# Spark erstellt hier einen Ordner mit part-Dateien, was für den Auto Loader perfekt ist.
(enrollments_df.write
               .mode("overwrite")
               .format("json")
               .save(source_data_path))
               
print(f"✅ Quelldaten wurden in den Auto Loader-Ordner '{source_data_path}' geschrieben.")
print("\n🚀 System ist bereit für den ersten Durchlauf der Medallion-Pipeline.")

✅ Quelldaten wurden in den Auto Loader-Ordner '/Volumes/workspace/default/volume/enrollments_raw' geschrieben.

🚀 System ist bereit für den ersten Durchlauf der Medallion-Pipeline.


In [0]:
# Prüfen ob die Ausgangslage gegeben ist:
# 1. Lese die JSON-Dateien aus dem Ordner in einen neuen DataFrame
check_df = spark.read.format("json").load(source_data_path)

# 2. Zeige den Inhalt des DataFrames direkt mit display() an
print("Inhalt des Quellordners für den Auto Loader:")
display(check_df)

Inhalt des Quellordners für den Auto Loader:


courses,enroll_id,quantity,student_id,timestamp,total
List(101),e001,1,1,2025-08-01T09:00:00,850
"List(102, 103)",e002,2,2,2025-08-02T10:00:00,1800
List(103),e003,1,3,2025-08-03T11:30:00,600


Nun benötigen wir noch Students

In [0]:
import pyspark.sql.functions as F

# --- Statische Lookup-Tabelle für Studenten ---
# Wir lesen die Daten aus der bereitgestellten students.json
students_lookup_df = spark.read.json(f"{volume_path}/students.json")

print("✅ Studenten-Lookup-Tabelle aus 'students.json' erstellt.")
# Optional: Anzeigen, um die geladenen Daten zu prüfen
display(students_lookup_df) 

✅ Studenten-Lookup-Tabelle aus 'students.json' erstellt.


email,gpa,profile,student_id,updated
anna@example.com,3.9,full-time,1,2025-08-01
ben@example.com,3.2,part-time,2,2025-08-03
clara@example.com,3.7,exchange,3,2025-08-06
